In [53]:
import pandas as pd
# The DataFrame 'df' is not displaying the headers correctly due to the initial row misalignment. 
# We need to adjust this first. Let's reload the data and skip the first row which seems to be incorrect.
df_corrected = pd.read_csv('raw_data/lex.csv', header=None)

# Now we define the first row as the header
df_corrected.columns = df_corrected.iloc[0]
df_corrected = df_corrected.drop(df_corrected.index[0])

# The headers now should be countries, let's transpose the data to make countries as one of the columns
df_transposed = df_corrected.set_index(df_corrected.columns[0]).T
df_transposed.reset_index(inplace=True)
df_transposed.rename(columns={df_transposed.columns[0]: 'year'}, inplace=True)

# Melt the transposed dataframe to get "country, year, GDP per capita" format
df_melted = pd.melt(df_transposed, id_vars=['year'], var_name='country', value_name='life_expectancy')

In [54]:
df_melted.dtypes

year               float64
country             object
life_expectancy    float64
dtype: object

In [55]:
df_melted

,year,country,life_expectancy
0,1800.0,Afghanistan,28.2
1,1801.0,Afghanistan,28.2
2,1802.0,Afghanistan,28.2
3,1803.0,Afghanistan,28.2
4,1804.0,Afghanistan,28.2
...,...,...,...
58991,2096.0,Zimbabwe,70.8
58992,2097.0,Zimbabwe,71.0
58993,2098.0,Zimbabwe,71.1
58994,2099.0,Zimbabwe,71.2


In [14]:
def reformatGDP(gdp):
    gdp_str = str(gdp).lower()
    if 'k' in gdp_str:
        new_str = gdp_str.replace('k', '')
        return float(new_str) * 1000
    return float(gdp)
    

In [ ]:
print(reformatGDP(187))

187.0


In [15]:
df_melted['life_expectancy'] = df_melted['life_expectancy'].map(reformatGDP)

In [16]:
df.to_csv('reformatted_data/reformatted_life_expectancy.csv', index=False)

In [58]:
df_melted.head()

,year,country,life_expectancy
0,1800.0,Afghanistan,28.2
1,1801.0,Afghanistan,28.2
2,1802.0,Afghanistan,28.2
3,1803.0,Afghanistan,28.2
4,1804.0,Afghanistan,28.2


In [3]:
import pandas as pd

In [14]:
df = pd.read_csv('reformatted_data/reformatted_life_expectancy.csv')

In [22]:
education_df2 = education_df[education_df['mean_number_of_years_in_school']>0]

In [23]:
education_df2.shape

(7000, 3)

In [24]:
education_df.shape

(7000, 3)

In [75]:
df.isnull().sum()

year              0
country           0
co2_per_capita    0
dtype: int64

In [23]:
value = '1800.0'
int_value = int(float(value))
print(int_value)

1800


In [32]:
df.rename(columns={'life_expectancy': 'gender_ratio_of_mean_years_in_school'}, inplace=True)

In [56]:
def date_to_int(year):
    return int(float(year))

In [59]:
df_melted['year'] = df_melted['year'].map(date_to_int)

In [70]:
df_2024 = df[df['year']==2024]

In [71]:
df_2024

,year,country,gdp_per_capita
224,2024,Afghanistan,1370.0
525,2024,Angola,5820.0
826,2024,Albania,16700.0
1127,2024,Andorra,55200.0
1428,2024,UAE,76900.0
...,...,...,...
57414,2024,Samoa,5650.0
57715,2024,Yemen,1720.0
58016,2024,South Africa,13400.0
58317,2024,Zambia,3370.0


In [72]:
df_2024.to_csv('2024ONLY_reformatted_gdp.csv', index=False)

##### Removing outliers for distribution plot 

In [4]:
import pandas as pd
import numpy as np

# Example dataset with a 'Value' column
df = pd.read_csv('reformatted_data/reformatted_gdp.csv')

# Function to remove outliers using the IQR method
def remove_outliers(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 2 * IQR
    
    # Filtering the DataFrame based on the IQR criteria
    filtered_df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return filtered_df

# Applying the function to remove outliers
filtered_data = remove_outliers(data, 'Value')

print(f"Original dataset size: {len(data)}")
print(f"Filtered dataset size: {len(filtered_data)}")


Original dataset size: 100
Filtered dataset size: 100


In [12]:
# To iterate through all the years from 1800 to 2024, remove outliers for each year,
# and then concatenate the results into a single DataFrame, follow these steps:

def remove_outliers_by_year_all(df, start_year=1800, end_year=2024):
    """
    Removes outliers for GDP per capita values outside the Interquartile Range (IQR)
    for each year within the specified range.
    
    Parameters:
    - df: DataFrame containing the columns 'year', 'country', and 'gdp_per_capita'
    - start_year: The start year of the range (inclusive)
    - end_year: The end year of the range (inclusive)
    
    Returns:
    - A DataFrame without outliers for the specified range of years
    """
    filtered_data_frames = []  # To store the filtered data for each year
    
    for year in range(start_year, end_year + 1):
        year_data = df[df['year'] == year]
        Q1 = year_data['co2_per_capita'].quantile(0.25)
        Q3 = year_data['co2_per_capita'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 3 * IQR
        upper_bound = Q3 + 3 * IQR
        filtered_year_data = year_data[(year_data['co2_per_capita'] >= lower_bound) & 
                                       (year_data['co2_per_capita'] <= upper_bound)]
        filtered_data_frames.append(filtered_year_data)
    
    # Concatenate all the yearly filtered data into one DataFrame
    filtered_data = pd.concat(filtered_data_frames)
    return filtered_data

# Applying the function to the entire dataset
gdp_df = pd.read_csv('reformatted_data/reformatted_co2.csv')
gdp_df['co2_per_capita'] = pd.to_numeric(gdp_df['co2_per_capita'], errors='coerce')
filtered_gdp_all_years = remove_outliers_by_year_all(gdp_df)

# Display the shape of the original and the filtered dataset to compare
original_shape = gdp_df.shape
filtered_shape = filtered_gdp_all_years.shape

original_shape, filtered_shape


((43262, 3), (39742, 3))

In [13]:
filtered_gdp_all_years.to_csv('reformatted_data/OUTLIERS_REMOVED_reformatted_co2.csv', index=False)

In [4]:
gdp_df = pd.read_csv('reformatted_data/reformatted_gdp.csv')

In [5]:
co2_df = pd.read_csv('reformatted_data/reformatted_CO2.csv')

In [6]:
life_expectancy_df = pd.read_csv('reformatted_data/reformatted_life_expectancy.csv')

In [7]:
education_df = pd.read_csv('reformatted_data/reformatted_gender_equality.csv')

In [15]:
final_merged_df2 .head()

,year,country,gdp_per_capita,life_expectancy,co2_per_capita,mean_number_of_years_in_school
0,1800,Afghanistan,599.0,28.2,0.001,NaN
1,1801,Afghanistan,599.0,28.2,0.001,NaN
2,1802,Afghanistan,599.0,28.2,0.001,NaN
3,1803,Afghanistan,599.0,28.2,0.001,NaN
4,1804,Afghanistan,599.0,28.2,0.001,NaN


In [17]:
# Merge the GDP and life expectancy DataFrames first
merged_df1 = pd.merge(gdp_df, life_expectancy_df, on=['country', 'year'], how='outer')

# Merge the resulting DataFrame with the CO2 emissions DataFrame
merged_df2 = pd.merge(merged_df1, co2_df, on=['country', 'year'], how='outer')

# Merge the resulting DataFrame with the education DataFrame
final_merged_df2 = pd.merge(merged_df2, education_df, on=['country', 'year'], how='outer')

# Display the first few rows of the final merged DataFrame
final_merged_df2.head()


,year,country,gdp_per_capita,life_expectancy,co2_per_capita,mean_number_of_years_in_school
0,1800,Afghanistan,599.0,28.2,0.001,NaN
1,1801,Afghanistan,599.0,28.2,0.001,NaN
2,1802,Afghanistan,599.0,28.2,0.001,NaN
3,1803,Afghanistan,599.0,28.2,0.001,NaN
4,1804,Afghanistan,599.0,28.2,0.001,NaN


In [11]:
final_merged_df2.to_csv('reformatted_data/reformatted_all.csv', index=False)

In [25]:
df = pd.read_csv('reformatted_data/reformatted_all.csv')

In [26]:
# Merge the GDP and life expectancy DataFrames first
merged_with_education_df = pd.merge(df, education_df, on=['country', 'year'], how='outer')

In [28]:
merged_with_education_df.to_csv('reformatted_data/reformatted_all_inc_education.csv', index=False)

In [19]:
education_df.isnull().sum()

year                              0
country                           0
mean_number_of_years_in_school    0
dtype: int64

In [19]:
df = df.dropna()

In [20]:
df.shape

(41986, 5)

In [21]:
df.to_csv('reformatted_data/reformatted_all.csv', index=False)

In [1]:
import pandas as pd

# Load the datasets
men_data_path = 'raw_data/mean_years_in_school_men_25_years_and_older.csv'
women_data_path = 'raw_data/mean_years_in_school_women_25_years_and_older.csv'

men_df = pd.read_csv(men_data_path)
women_df = pd.read_csv(women_data_path)

# Display the first few rows of each dataset
men_df.head(), women_df.head()


(       country  1970  1971  1972  1973  1974  1975  1976  1977  1978  ...  \
 0  Afghanistan   0.7   0.7   0.8   0.8   0.8   0.9   0.9   0.9   1.0  ...   
 1       Angola   1.4   1.5   1.5   1.6   1.7   1.7   1.8   1.9   2.0  ...   
 2      Albania   5.1   5.2   5.3   5.5   5.6   5.7   5.9   6.0   6.1  ...   
 3          UAE   3.0   3.1   3.2   3.3   3.5   3.6   3.8   3.9   4.1  ...   
 4    Argentina   5.9   6.0   6.1   6.2   6.3   6.4   6.6   6.7   6.8  ...   
 
    2000  2001  2002  2003  2004  2005  2006  2007  2008  2009  
 0   2.1   2.1   2.2   2.3   2.3   2.4   2.4   2.5   2.6   2.6  
 1   4.0   4.1   4.3   4.4   4.5   4.6   4.7   4.9   5.0   5.1  
 2   9.1   9.2   9.4   9.5   9.6   9.8   9.9  10.0  10.1  10.2  
 3   8.0   8.2   8.4   8.5   8.7   8.8   9.0   9.2   9.3   9.5  
 4   8.9   9.0   9.1   9.2   9.3   9.4   9.5   9.6   9.7   9.8  
 
 [5 rows x 41 columns],
        country  1970  1971  1972  1973  1974  1975  1976  1977  1978  ...  \
 0  Afghanistan   0.0   0.1   0.1   

In [4]:
# Melt the dataframes to long format
men_long = men_df.melt(id_vars=['country'], var_name='year', value_name='mean_years_men')
women_long = women_df.melt(id_vars=['country'], var_name='year', value_name='mean_years_women')

# Merge the datasets on country and year
merged_df = pd.merge(men_long, women_long, on=['country', 'year'])

# Calculate the mean number of years in school for men and women
merged_df['mean_years_avg'] = (merged_df['mean_years_men'] + merged_df['mean_years_women']) / 2

# Select and rename columns as requested
result_df = merged_df[['year', 'country', 'mean_years_avg']]
result_df.rename(columns={'mean_years_avg': 'mean_number_of_years_in_school'}, inplace=True)

# Display the first few rows of the result
result_df.head()


/var/folders/z7/qx60p0293lsfh_9g6rxq9c740000gn/T/ipykernel_3140/3028728516.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(columns={'mean_years_avg': 'mean_number_of_years_in_school'}, inplace=True)


,year,country,mean_number_of_years_in_school
0,1970,Afghanistan,0.35
1,1970,Angola,0.95
2,1970,Albania,4.50
3,1970,UAE,2.25
4,1970,Argentina,5.70


In [5]:
result_df.to_csv('reformatted_data/reformatted_gender_equality.csv', index=False)

In [29]:
import pandas as pd

# Replace this with the path to your locally saved CSV file
file_path = 'reformatted_data/reformatted_life_expectancy.csv'

# Load the CSV file
data = pd.read_csv(file_path)

# Assuming 'country' is the column with the country names
countries = data['country'].unique()

# Base URL for the CIA World Factbook country pages
base_url = "https://www.cia.gov/the-world-factbook/countries/"

# Create a new DataFrame for country links
country_links = pd.DataFrame({
    'country': countries,
    'link': [base_url + country.lower().replace(' ', '-') for country in countries]
})

# Save to a new CSV file
country_links.to_csv('country_factbook_links.csv', index=False)
